In [ ]:
import os
import pickle
def make_index(file):
    origins = []
    current_origin = 0

    original_position = file.tell()
    file.seek(0)

    while True:
        try:
            pickle.load(file)
            origins.append(current_origin)
            current_origin = file.tell()
        except EOFError:
            break
        except Exception as e:
            print(f"Exception: {e} in daqfile initializing index of file")

    #print(f"Make index {type(file)}")
    file.seek(original_position)

    return origins


filename = 'fixedWhiteBloodCellsShort.npy'

try:
    file = open(filename, 'r+b')
except Exception as e:
    print(f"Exception: {e} in daqfile for playback")
    raise

origins = make_index(file)

In [ ]:
#COUNT
if(origins is None):
    raise Exception("Daqfile: file has not been initialized")
len(origins)

In [ ]:
#GET
# file.seek(origins[0])
# packet = pickle.load(file)
data = []
for i in range(6):
    file.seek(origins[i])
    packet = pickle.load(file)
    data.append(packet['event']['data']['adc_data'])

In [ ]:
# packet = file.get(record_n)
data1 = packet['event']['data']['adc_data']
print(data1)

Comparing with Valery Software

Numbers to match or get close to\
471476.88671875 \
717160.87109375 \
726431.96484375 \
742656.37890625 \
744974.15234375 \
1062509.11328125 \
Median gets closest so far


In [ ]:
import torch
import time
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(6):
    bigarr1 = torch.from_numpy(data[i].astype(float)).to(device)
    # bigarr1 = torch.from_numpy(savgol_filter(data[i], window_length=int(len(data[i][0])/10), polyorder=1, axis=1).astype(float)).to(device)

    # a1 = torch.std_mean(bigarr1, dim=0, keepdim=True)
    a1 = torch.std(bigarr1, dim=0, keepdim=True)
    b1 = torch.mean(bigarr1, dim=0).to(device)
    #Height
    e1 = (bigarr1.amax(dim=0) - bigarr1.amin(dim=0)).to(device)
    # e1 = (bigarr1.amax(dim=1).to(device) - bigarr1.amin(dim=1).to(device)) 
    # f1 = (bigarr1.lt(b1 - (a1*0.3)) * bigarr1).to(device)
    f1 = (bigarr1.lt(b1) * bigarr1).to(device)
    #WITDH
    g1 = f1.count_nonzero(dim=0).to(device)
    #AREA
    h1 = ((b1 * g1) - f1.sum(dim=0)).to(device)

    # print(a1)
    print("Height")
    print(e1.cpu().tolist())
    print("Width")
    print(g1.cpu().tolist())
    print("Area")
    print(h1.cpu().tolist())

In [ ]:
import torch
import time
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(6):
    bigarr1 = torch.from_numpy(data[i].astype(float)).to(device)
    # bigarr1 = torch.from_numpy(savgol_filter(data[i], window_length=int(data[i][0].__len__()/10), polyorder=1, axis=1).astype(float)).to(device)

    a1 = torch.std(bigarr1, dim=0, keepdim=True).to(device)
    b1 = torch.median(bigarr1, dim=0)
    e1 = (bigarr1.amax(dim=0) - bigarr1.amin(dim=0)).to(device)
    # e1 = (bigarr1.amax(dim=1).to(device) - bigarr1.amin(dim=1).to(device)) 
    f1 = (bigarr1.lt(b1[0]) * bigarr1).to(device)
    #WITDH
    g1 = f1.count_nonzero(dim=0).to(device)
    #AREA
    h1 = (((b1[0]) * g1) - f1.sum(dim=0)).to(device)

    # print(a1)
    print("Height")
    print(e1.cpu().tolist())
    print("Width")
    print(g1.cpu().tolist())
    print("Area")
    print(h1.cpu().tolist())

In [3]:
a = [1,2,3,4,5,6,7,8,9]
del(a[-1])
del(a[-2])
print(a)

[1, 2, 3, 4, 5, 6, 8]


In [5]:
WRITE = False
WRITE = False if WRITE else True
print(WRITE)

In [4]:
import numpy as np

scale = np.logspace(start=0, stop=7, num = 20)
print(scale)

[1.00000000e+00 2.33572147e+00 5.45559478e+00 1.27427499e+01
 2.97635144e+01 6.95192796e+01 1.62377674e+02 3.79269019e+02
 8.85866790e+02 2.06913808e+03 4.83293024e+03 1.12883789e+04
 2.63665090e+04 6.15848211e+04 1.43844989e+05 3.35981829e+05
 7.84759970e+05 1.83298071e+06 4.28133240e+06 1.00000000e+07]
